# Présentation

Dans ce code nous utilisons l'ancien modèle pour prédire de façon plus fine le rendement d'une parcelle.

# Importation

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.utils import shuffle
# fix random seed for reproducibility
tf.random.set_seed(7)
# pip3 install tensorflow numpy pandas scikit-learn

# Notebook
from IPython.display import clear_output
clear_output()

# Importation des données

In [2]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df2 = pd.read_csv("Crop_Yield_Data_challenge_2.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)

# Les données dans cette cellule permet de normaliser les entrées entre [0, 1]
borne = [
    (-0.0440750, 1.1379025),
    (-0.0215250, 1.1155175),
    (-0.1473925, 1.13914),
    (0.04604250, 1.2140225),
    (21762.0000, 56916.0),
    (293.0, 48423.0)
]
mini = np.array([
    -0.0440750,
    -0.0215250,
    -0.1473925,
    0.04604250,
    21762.0000,
    293.0,
])
multi = np.array([
    1.1819775, 
    1.1370425, 
    1.2865325, 
    1.16798, 
    35154.0, 
    48130.0
])

# Entrainement
x2 = []
y2 = []
for i in range(len(df2)):
    with open(f"inputs_lstm/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x2.append(arr)
    # Normalisation des sorties
    y2.append([(df2.loc[i, "Rice Yield (kg/ha)"]-5200)/(8000-5200)])
    

x2 = pad_sequences(x2, dtype="float", maxlen=32)
y2 = np.array(y2)
x2, y2 = shuffle(x2, y2)

# Création du modèle

In [3]:
model_reg = Sequential()
# Utilisation du modèle précédent
model = load_model("models_finaux/classification/14-1.00.h5")
model_reg.add(Input(shape=(None, 100, 100, 6)))
for layer in model.layers[:-1]:
    layer.trainable = False
    model_reg.add(layer)

# Ajout de couches de sorties
model_reg.add(Dense(16, name="transfert_dense1"))
model_reg.add(Dense(16, name="transfert_dense2"))
model_reg.add(Dense(1 , name="transfert_dense3", activation="sigmoid"))

clear_output()
model_reg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 98, 98, 3)         984       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 2048)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 32)                262176    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                        

In [4]:
# Compilation
model_reg.compile(
    optimizer='sgd',
    loss='mse',
    metrics=['mse']
)

# Entrainement

In [5]:
checkpoint = ModelCheckpoint(
    filepath='models_finaux/regression/{epoch:02d}-{val_mse:.5f}.h5', 
    monitor="val_mse", mode="min", 
    save_best_only=True, verbose=1
)
callbacks = [checkpoint]

In [6]:
history = model_reg.fit(
    x2, y2,
    batch_size=16, 
    validation_split = 0.1,
    epochs=3, callbacks=callbacks
)
# Les données ci-dessous ne sont pas représentatif du vrai entrainement fait sur une VM pendant plusieures heures

Epoch 1/3
1/1 [==============================] - ETA: 0s - loss: 0.1010 - mse: 0.1010
Epoch 1: val_mse improved from inf to 0.00321, saving model to models_finaux/regression/01-0.00321.h5
1/1 [==============================] - 6s 6s/step - loss: 0.1010 - mse: 0.1010 - val_loss: 0.0032 - val_mse: 0.0032
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 0.0741 - mse: 0.0741
Epoch 2: val_mse did not improve from 0.00321
1/1 [==============================] - 0s 444ms/step - loss: 0.0741 - mse: 0.0741 - val_loss: 0.0060 - val_mse: 0.0060
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 0.0713 - mse: 0.0713
Epoch 3: val_mse did not improve from 0.00321
1/1 [==============================] - 0s 383ms/step - loss: 0.0713 - mse: 0.0713 - val_loss: 0.0050 - val_mse: 0.0050
